# CIFAR-10 Challange - 김영인

1. wide resnet 101-2
2. \+ Data Augmentation(RandomResizedCrop, RandomHorizontalFlip)
3. \+ Data Normalize
5. \+ Optimizer(momentum, scheduler)


In [4]:
import random

import torch
import numpy as np
from matplotlib import pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, datasets

random_seed = 4332
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

device0 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device1 = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

device = device0
print(f"device: {device}") if torch.cuda.is_available() else print("device: cpu")

device: cuda:0


## Hyper parameter Setting 

In [5]:
learning_rate = 0.1
scheduler_step = 30
scheduler_gamma = 0.2
training_epochs = 100
batch_size = 32
momentum = 0.9

## Load & Preprocess Data

In [6]:
transform_train = transforms.Compose(
    [transforms.RandomResizedCrop(224), # data augmentation, 224: image size, ImageNet pretrained model에 맞추기 위해서 224 size로 설정
     transforms.RandomHorizontalFlip(), # data augmentation, 좌우로 대칭
     transforms.ToTensor(), # numpy array를 pytorch tensor로 바꿔주는 역할
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)) # dataset의 mean, std를 이용해서 -1~1 로 normalize
    ])

transform_test = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = datasets.CIFAR10(root='./data', train=True,
                                      download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=batch_size,
                                         shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Load and Define pretrained model and fine-tuning

In [7]:
print(models.wide_resnet101_2(pretrained=True).fc)

Linear(in_features=2048, out_features=1000, bias=True)


In [8]:
class WideResNet(nn.Module):
    def __init__ (self):
        super(WideResNet, self).__init__()
        self.resnet = models.wide_resnet101_2(pretrained=True)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, 10)
        
    def forward(self, x):
        x = self.resnet(x)
        return x

## Load Model

In [9]:
net = WideResNet()
net = net.to(device)

criterion = nn.CrossEntropyLoss() 
optimizer = optim.SGD(net.parameters(), 
                      lr=learning_rate,
                      momentum=momentum)
scheduler = optim.lr_scheduler.StepLR(optimizer, 
                                      step_size = scheduler_step,
                                      gamma = scheduler_gamma)

## Train

In [10]:
net.train()

for epoch in range(training_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()

        outputs = net(inputs) 
        loss =  criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
            
        if i == 780:
            class_correct = np.zeros(10)
            class_total = np.zeros(10)

            net.eval()

            with torch.no_grad():
                for data in testloader:
                    images, labels = data
                    images = images.to(device)
                    labels = labels.to(device)
                    outputs = net(images)
                    _, predicted = torch.max(outputs, 1)
                    c = (predicted == labels).squeeze()
                    for i in range(4):
                        label = labels[i]
                        class_correct[label] += c[i].item()
                        class_total[label] += 1
            print('==================')
            print('Mean Accuracy : ', 100*np.sum(class_correct)/np.sum(class_total))
            print('==================')
            
            net.train()
        
print('Finished Training')

[1,   100] loss: 3.672
[1,   200] loss: 2.261
[1,   300] loss: 2.203
[1,   400] loss: 2.115
[1,   500] loss: 2.088
[1,   600] loss: 2.079
[1,   700] loss: 2.066
Mean Accuracy :  23.9
[1,   800] loss: 2.074
[1,   900] loss: 2.036
[1,  1000] loss: 2.014
[1,  1100] loss: 1.990
[1,  1200] loss: 2.020
[1,  1300] loss: 1.965
[1,  1400] loss: 1.979
[1,  1500] loss: 1.947
[2,   100] loss: 1.915
[2,   200] loss: 1.949
[2,   300] loss: 1.921
[2,   400] loss: 1.921
[2,   500] loss: 1.888
[2,   600] loss: 1.888
[2,   700] loss: 1.880
Mean Accuracy :  30.73
[2,   800] loss: 1.849
[2,   900] loss: 1.845
[2,  1000] loss: 1.808
[2,  1100] loss: 1.826
[2,  1200] loss: 1.809
[2,  1300] loss: 1.808
[2,  1400] loss: 1.796
[2,  1500] loss: 1.807
[3,   100] loss: 1.749
[3,   200] loss: 1.722
[3,   300] loss: 1.743
[3,   400] loss: 1.729
[3,   500] loss: 1.710
[3,   600] loss: 1.693
[3,   700] loss: 1.702
Mean Accuracy :  39.75
[3,   800] loss: 1.687
[3,   900] loss: 1.665
[3,  1000] loss: 1.675
[3,  1100] l

[20,  1300] loss: 0.648
[20,  1400] loss: 0.665
[20,  1500] loss: 0.615
[21,   100] loss: 0.670
[21,   200] loss: 0.682
[21,   300] loss: 0.632
[21,   400] loss: 0.664
[21,   500] loss: 0.662
[21,   600] loss: 0.683
[21,   700] loss: 0.673
Mean Accuracy :  76.28
[21,   800] loss: 0.680
[21,   900] loss: 0.645
[21,  1000] loss: 0.664
[21,  1100] loss: 0.667
[21,  1200] loss: 0.647
[21,  1300] loss: 0.662
[21,  1400] loss: 0.651
[21,  1500] loss: 0.648
[22,   100] loss: 0.637
[22,   200] loss: 0.652
[22,   300] loss: 0.660
[22,   400] loss: 0.627
[22,   500] loss: 0.662
[22,   600] loss: 0.644
[22,   700] loss: 0.647
Mean Accuracy :  74.77
[22,   800] loss: 0.628
[22,   900] loss: 0.647
[22,  1000] loss: 0.626
[22,  1100] loss: 0.686
[22,  1200] loss: 0.644
[22,  1300] loss: 0.660
[22,  1400] loss: 0.666
[22,  1500] loss: 0.644
[23,   100] loss: 0.657
[23,   200] loss: 0.618
[23,   300] loss: 0.643
[23,   400] loss: 0.637
[23,   500] loss: 0.620
[23,   600] loss: 0.579
[23,   700] loss: 

[40,   700] loss: 0.465
Mean Accuracy :  79.71
[40,   800] loss: 0.447
[40,   900] loss: 0.470
[40,  1000] loss: 0.470
[40,  1100] loss: 0.449
[40,  1200] loss: 0.505
[40,  1300] loss: 0.445
[40,  1400] loss: 0.461
[40,  1500] loss: 0.502
[41,   100] loss: 0.446
[41,   200] loss: 0.464
[41,   300] loss: 0.462
[41,   400] loss: 0.456
[41,   500] loss: 0.459
[41,   600] loss: 0.449
[41,   700] loss: 0.466
Mean Accuracy :  81.51
[41,   800] loss: 0.489
[41,   900] loss: 0.479
[41,  1000] loss: 0.480
[41,  1100] loss: 0.453
[41,  1200] loss: 0.448
[41,  1300] loss: 0.502
[41,  1400] loss: 0.470
[41,  1500] loss: 0.485
[42,   100] loss: 0.437
[42,   200] loss: 0.445
[42,   300] loss: 0.455
[42,   400] loss: 0.450
[42,   500] loss: 0.435
[42,   600] loss: 0.429
[42,   700] loss: 0.478
Mean Accuracy :  80.51
[42,   800] loss: 0.423
[42,   900] loss: 0.469
[42,  1000] loss: 0.466
[42,  1100] loss: 0.420
[42,  1200] loss: 0.466
[42,  1300] loss: 0.454
[42,  1400] loss: 0.463
[42,  1500] loss: 0

[59,  1300] loss: 0.350
[59,  1400] loss: 0.376
[59,  1500] loss: 0.388
[60,   100] loss: 0.372
[60,   200] loss: 0.368
[60,   300] loss: 0.376
[60,   400] loss: 0.381
[60,   500] loss: 0.352
[60,   600] loss: 0.351
[60,   700] loss: 0.394
Mean Accuracy :  82.9
[60,   800] loss: 0.392
[60,   900] loss: 0.369
[60,  1000] loss: 0.386
[60,  1100] loss: 0.351
[60,  1200] loss: 0.372
[60,  1300] loss: 0.377
[60,  1400] loss: 0.377
[60,  1500] loss: 0.395
[61,   100] loss: 0.342
[61,   200] loss: 0.352
[61,   300] loss: 0.345
[61,   400] loss: 0.353
[61,   500] loss: 0.368
[61,   600] loss: 0.352
[61,   700] loss: 0.354
Mean Accuracy :  82.3
[61,   800] loss: 0.356
[61,   900] loss: 0.359
[61,  1000] loss: 0.350
[61,  1100] loss: 0.373
[61,  1200] loss: 0.372
[61,  1300] loss: 0.347
[61,  1400] loss: 0.386
[61,  1500] loss: 0.358
[62,   100] loss: 0.348
[62,   200] loss: 0.364
[62,   300] loss: 0.356
[62,   400] loss: 0.366
[62,   500] loss: 0.398
[62,   600] loss: 0.356
[62,   700] loss: 0.

[79,   700] loss: 0.316
Mean Accuracy :  83.96
[79,   800] loss: 0.301
[79,   900] loss: 0.321
[79,  1000] loss: 0.335
[79,  1100] loss: 0.303
[79,  1200] loss: 0.315
[79,  1300] loss: 0.317
[79,  1400] loss: 0.312
[79,  1500] loss: 0.325
[80,   100] loss: 0.287
[80,   200] loss: 0.327
[80,   300] loss: 0.334
[80,   400] loss: 0.330
[80,   500] loss: 0.330
[80,   600] loss: 0.339
[80,   700] loss: 0.315
Mean Accuracy :  84.11
[80,   800] loss: 0.311
[80,   900] loss: 0.310
[80,  1000] loss: 0.318
[80,  1100] loss: 0.321
[80,  1200] loss: 0.325
[80,  1300] loss: 0.299
[80,  1400] loss: 0.327
[80,  1500] loss: 0.306
[81,   100] loss: 0.301
[81,   200] loss: 0.288
[81,   300] loss: 0.331
[81,   400] loss: 0.293
[81,   500] loss: 0.318
[81,   600] loss: 0.299
[81,   700] loss: 0.290
Mean Accuracy :  84.38
[81,   800] loss: 0.317
[81,   900] loss: 0.311
[81,  1000] loss: 0.300
[81,  1100] loss: 0.334
[81,  1200] loss: 0.310
[81,  1300] loss: 0.298
[81,  1400] loss: 0.341
[81,  1500] loss: 0

[98,  1300] loss: 0.267
[98,  1400] loss: 0.268
[98,  1500] loss: 0.290
[99,   100] loss: 0.263
[99,   200] loss: 0.256
[99,   300] loss: 0.271
[99,   400] loss: 0.293
[99,   500] loss: 0.279
[99,   600] loss: 0.305
[99,   700] loss: 0.271
Mean Accuracy :  85.02
[99,   800] loss: 0.269
[99,   900] loss: 0.298
[99,  1000] loss: 0.277
[99,  1100] loss: 0.285
[99,  1200] loss: 0.279
[99,  1300] loss: 0.297
[99,  1400] loss: 0.278
[99,  1500] loss: 0.294
[100,   100] loss: 0.281
[100,   200] loss: 0.281
[100,   300] loss: 0.263
[100,   400] loss: 0.279
[100,   500] loss: 0.271
[100,   600] loss: 0.270
[100,   700] loss: 0.272
Mean Accuracy :  84.82
[100,   800] loss: 0.283
[100,   900] loss: 0.262
[100,  1000] loss: 0.267
[100,  1100] loss: 0.258
[100,  1200] loss: 0.288
[100,  1300] loss: 0.276
[100,  1400] loss: 0.292
[100,  1500] loss: 0.267
Finished Training


## Test

In [11]:
class_correct = np.zeros(10)
class_total = np.zeros(10)

net.eval()

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2f %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
print('Mean Accuracy : ', 100*np.sum(class_correct)/np.sum(class_total))

Accuracy of plane : 85.400000 %
Accuracy of   car : 88.500000 %
Accuracy of  bird : 80.700000 %
Accuracy of   cat : 63.500000 %
Accuracy of  deer : 86.000000 %
Accuracy of   dog : 87.400000 %
Accuracy of  frog : 90.500000 %
Accuracy of horse : 88.400000 %
Accuracy of  ship : 89.400000 %
Accuracy of truck : 92.000000 %
Mean Accuracy :  85.18
